In [45]:
import os

from scipy import ndimage
import math
import json
import torch
import numpy as np

import cv2
import matplotlib

from matplotlib import pyplot as plt

from tqdm import tqdm
from pathlib import Path
from dotenv import load_dotenv

from utils.yolo import get_teeth_ROI
from utils.edge import tooth_isolation, gum_jaw_separation, vertical_separation, bounding_teeth_on_origin
from utils.preprocess import recovery_rotated_bounding, xyxy2xywh, get_image_by_labels, get_labels_by_image
from utils.preprocess import xyxy_reformat, rotate_bounding_boxes, rect_include_another

matplotlib.use('module://matplotlib_inline.backend_inline')
load_dotenv()


True

In [46]:
target_labels = ['caries', 'endo', 'post', 'crown']
image_labels_df = get_image_by_labels(target_labels)[target_labels]

image_labels_df


,caries,endo,post,crown
filename,,,,
00006145,True,False,False,False
00008026,True,True,True,True
00008113,True,True,True,True
00008117,True,False,False,False
00008131,True,True,False,False
...,...,...,...,...
202012050412159442018_0690727A,False,True,True,True
202012050412229432017_0781002A,False,True,False,False
202012051012069362000_0760503A,False,True,False,True


In [47]:
data_dir = Path('..') / '..' / 'Datasets' / 'phase-2'

filename = image_labels_df.index[1]
filepath_image = data_dir / f'{filename}.jpg'
filepath_json = data_dir / f'{filename}.json'


In [48]:
tooth_position_dict = {
    0: 'left',
    1: 'middle',
    2: 'right'
}

model = torch.hub.load(r'..\YOLO', 'custom', path=r'..\YOLO\weights\8-bound.pt', source='local')
# Image
# Inference
results = model(filepath_image)

teeth_roi = get_teeth_ROI(results)
teeth_roi_images = teeth_roi['images'][filename]
teeth_roi_split_teeth = teeth_roi['split_teeth']
teeth_roi_images


YOLOv5  torch 1.10.0+cpu CPU

Fusing layers... 
Model summary: 270 layers, 7041205 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


requirements: C:\Users\denni\anaconda3\envs\dentist-cv\Lib\site-packages\yolov5\requirements.txt not found, check failed.


[{'flag': 'upper',
  'number': 0,
  'org_file_name': '00008026',
  'offset': array([698, 281]),
  'image': array([[[129, 129, 129],
          [130, 130, 130],
          [133, 133, 133],
          ...,
          [135, 135, 135],
          [133, 133, 133],
          [131, 131, 131]],
  
         [[127, 127, 127],
          [129, 129, 129],
          [133, 133, 133],
          ...,
          [138, 138, 138],
          [139, 139, 139],
          [139, 139, 139]],
  
         [[127, 127, 127],
          [132, 132, 132],
          [136, 136, 136],
          ...,
          [145, 145, 145],
          [145, 145, 145],
          [146, 146, 146]],
  
         ...,
  
         [[142, 142, 142],
          [143, 143, 143],
          [137, 137, 137],
          ...,
          [117, 117, 117],
          [120, 120, 120],
          [122, 122, 122]],
  
         [[142, 142, 142],
          [139, 139, 139],
          [130, 130, 130],
          ...,
          [115, 115, 115],
          [116, 116, 116],
    

# One file check

In [49]:
image_labels = {}
labels = get_labels_by_image(filepath_json, target_labels)
for target_roi in teeth_roi_images:
    target_roi_image = target_roi['image']
    flag = target_roi['flag']
    tooth_position = tooth_position_dict[target_roi['number']]
    im_g = cv2.cvtColor(target_roi_image, cv2.COLOR_RGBA2GRAY)
    im_g_shape = np.array(np.array(im_g.shape)[[1, 0]])

    isolation_data = tooth_isolation(im_g, flag=flag, tooth_position=tooth_position, rotation_fix=False)
    if not isolation_data:
        continue
    regions = isolation_data['crop_regions']
    theta = isolation_data['angle']
    offset = target_roi['offset']

    phi = math.radians(theta)
    for label in labels:
        xyxy = np.hstack(label['points'])  # [x, y, x, y]
        xyxy = xyxy_reformat(np.array([xyxy]))

        xyxy = xyxy - np.tile(offset, 2)
        if xyxy.min() < 0:
            continue

        xyxy = rotate_bounding_boxes(phi, im_g_shape, xyxy)
        xyxy = xyxy[0].astype(int)

        for tooth_number, region in regions.items():
            tooth_xyxy = region['xyxy']
            if rect_include_another(tooth_xyxy, xyxy) > 0.5:
                key = f'{filename}-{tooth_number}'
                if not key in image_labels.keys():
                    image_labels[key] = []
                image_labels[key].append(label['label'])

image_labels


{'00008026-11': ['post', 'endo', 'crown'],
 '00008026-21': ['post', 'endo', 'crown'],
 '00008026-12': ['crown'],
 '00008026-22': ['crown'],
 '00008026-26': ['crown'],
 '00008026-46': ['caries'],
 '00008026-32': ['crown'],
 '00008026-36': ['crown', 'post', 'endo']}

In [50]:
image_labels = {}
for filename in tqdm(image_labels_df.index):
    filepath_image = data_dir / f'{filename}.jpg'
    filepath_json = data_dir / f'{filename}.json'

    results = model(filepath_image)

    teeth_roi = get_teeth_ROI(results)
    teeth_roi_images = teeth_roi['images'][filename]
    teeth_roi_split_teeth = teeth_roi['split_teeth']

    labels = get_labels_by_image(filepath_json, target_labels)
    for target_roi in teeth_roi_images:
        target_roi_image = target_roi['image']
        flag = target_roi['flag']
        tooth_position = tooth_position_dict[target_roi['number']]
        im_g = cv2.cvtColor(target_roi_image, cv2.COLOR_RGBA2GRAY)
        im_g_shape = np.array(np.array(im_g.shape)[[1, 0]])

        isolation_data = tooth_isolation(im_g, flag=flag, tooth_position=tooth_position, rotation_fix=False)
        if not isolation_data:
            continue
        regions = isolation_data['crop_regions']
        theta = isolation_data['angle']
        offset = target_roi['offset']

        phi = math.radians(theta)
        for label in labels:
            xyxy = np.hstack(label['points'])  # [x, y, x, y]
            xyxy = xyxy_reformat(np.array([xyxy]))

            xyxy = xyxy - np.tile(offset, 2)
            if xyxy.min() < 0:
                continue

            xyxy = rotate_bounding_boxes(phi, im_g_shape, xyxy)
            xyxy = xyxy[0].astype(int)

            for tooth_number, region in regions.items():
                tooth_xyxy = region['xyxy']
                if rect_include_another(tooth_xyxy, xyxy) > 0.5:
                    key = f'{filename}-{tooth_number}'
                    if not key in image_labels.keys():
                        image_labels[key] = []
                    image_labels[key].append(label['label'])


  3%|▎         | 12/365 [01:28<29:00,  4.93s/it] 

00008155:dict_keys([])
00008155:dict_keys([])


  7%|▋         | 26/365 [03:21<43:01,  7.61s/it]

00008218:dict_keys(['47', '33', '43'])


  8%|▊         | 29/365 [03:34<28:27,  5.08s/it]

...

202012031112019181962_0270118A:dict_keys(['33', '13', '23', '27'])


 98%|█████████▊| 357/365 [47:51<00:42,  5.25s/it]

202012040612241941985_0530925A:dict_keys([])
202012040612241941985_0530925A:dict_keys([])


100%|██████████| 365/365 [49:06<00:00,  8.07s/it]

202012051012279382005_0621018A:dict_keys([])
202012051012279382005_0621018A:dict_keys([])


In [51]:
# for tooth_number, r in region.items():
#     print(r)
# target_roi
# label
# !jupyter nbconvert --to script prepare_classify_format.ipynb

j = json.dumps(image_labels)

with open('image_labels_for_classify.json', 'w') as f:
    f.write(j)


## Check json file and jpg file pair

In [52]:
jpgs = list(data_dir.glob('*.jpg'))
jsons = list(data_dir.glob('*.json'))

jpgs_set = {jpg.stem for jpg in jpgs}
jsons_set = {json_file.stem for json_file in jsons}

(jpgs_set - jsons_set) | (jsons_set - jpgs_set)


set()